In [1]:
!pip install keras-tuner

     |████████████████████████████████| 71kB 5.1MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.2-cp37-none-any.whl size=78938 sha256=1deb97b9d09f3b81ee290503fbef5c3d8fb6fecf9fa76a62d28d5c9bbb0ed31e
  Stored in directory: /root/.cache/pip/wheels/bb/a1/8a/7c3de0efb3707a1701b36ebbfdbc4e67aedf6d4943a1f463d6
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp37-none-any.whl size=15356 sha256=2a74d81afda7dbe4c9599b93f80cc83bdcb9de8d6b118964bd7f5195f7c934e9
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [3]:
fashion_mnist=keras.datasets.fashion_mnist

In [7]:
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

In [8]:
train_images=train_images/255.0
test_images=test_images/255.0

In [9]:
train_images[0].shape

(28, 28)

In [11]:
train_images = train_images.reshape(len(train_images),28,28,1)
test_images = test_images.reshape(len(test_images),28,28,1)

In [32]:
def build_model(hp):
  model = keras.Sequential([
keras.layers.Conv2D(
filters=hp.Int('conv_1_filter',min_value=32, max_value=128, step=16),
kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
activation='relu',
input_shape=(28,28,1)
),
keras.layers.Conv2D(
filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
 kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
activation='relu'
),
keras.layers.Flatten(),
keras.layers.Dense(
units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
activation='relu'
),
keras.layers.Dense(10, activation='softmax')#output layer
  ])
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
  
  return model

In [33]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [35]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='output',project_name="Mnist Fashion")

In [ ]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Trial 4 Complete [00h 11m 22s]
val_accuracy: 0.8550000190734863

Best val_accuracy So Far: 0.8691666722297668
Total elapsed time: 00h 24m 40s

Search: Running Trial #5

Hyperparameter    |Value             |Best Value So Far 
conv_1_filter     |112               |96                
conv_1_kernel     |3                 |5                 
conv_2_filter     |64                |32                
conv_2_kernel     |5                 |3                 
dense_1_units     |112               |48                
learning_rate     |0.01              |0.01              

Epoch 1/3
1688/1688 [==============================] - 645s 382ms/step - loss: 2.3270 - accuracy: 0.0982 - val_loss: 2.3050 - val_accuracy: 0.1008
Epoch 2/3
1313/1688 [======================>.......] - ETA: 2:19 - loss: 2.3040 - accuracy: 0.1004

In [ ]:
model=tuner_search.get_best_models(num_models=1)[0]

In [ ]:
model.summary()